In [ ]:
pip install xgboost-ray

In [ ]:
import ray
from ray.air.config import ScalingConfig, RunConfig
from ray.train.xgboost import XGBoostTrainer
from ray.train.xgboost import XGBoostPredictor
from ray import tune
from ray.tune import Tuner, TuneConfig
from ray import serve
import requests, json
from starlette.requests import Request
import numpy as np

# Ray libraries

Ray includes set of high-level easy-to-use APIs for
ingesting data, training models – including reinforcement learning
models – tuning those models and then serving them.

<img src="https://technical-training-assets.s3.us-west-2.amazonaws.com/Introduction_to_Ray_AIR/e2e_air.png" width=600 loading="lazy"/>

1. [**Ray Data**](https://docs.ray.io/en/latest/data/dataset.html)  
Scalable, framework-agnostic data loading and transformation across training, tuning, and prediction.

2. [**Ray Train**](https://docs.ray.io/en/latest/train/train.html)  
Distributed multi-node and multi-core model training with fault tolerance that integrates with popular machine learning training libraries.

3. [**Ray Tune**](https://docs.ray.io/en/latest/tune/index.html)  
Scales hyperparameter tuning to optimize model performance.

4. [**Ray Serve**](https://docs.ray.io/en/latest/serve/index.html)  
Deploys a model or ensemble of models for online inference.

5. [**Ray RLlib**](https://docs.ray.io/en/latest/rllib/index.html)  
Scales reinforcement learning workloads that integrate with other Ray AIR libraries; not covered in-depth in this module

Key principles behind Ray and its libraries are
* Performance
* Developer experience and simplicity

## Quick end-to-end example

|Ray AIR Component|NYC Taxi Use Case|
|:--|:--|
|Ray Data|Use `Preprocessor` to load and transform input data.|
|Ray Train|Use `Trainer` to scale XGBoost model training.|
|Ray Tune|Use `Tuner` for hyperparameter search.|
|Ray AIR Predictor|Use `BatchPredictor` to load model from best checkpoint for batch inference; part of Ray Train.|
|Ray Serve|Use `PredictorDeployment` for online inference.|

For this classification task, you will apply a simple [XGBoost](https://xgboost.readthedocs.io/en/stable/) (a gradient boosted trees framework) model to the June 2021 [New York City Taxi & Limousine Commission's Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This dataset contains over 2 million samples of yellow cab rides, and the goal is to predict whether a trip will result in a tip greater than 20% or not.

**Dataset features**
* **`passenger_count`**
    * Float (whole number) representing number of passengers.
* **`trip_distance`** 
    * Float representing trip distance in miles.
* **`fare_amount`**
    * Float representing total price including tax, tip, fees, etc.
* **`trip_duration`**
    * Integer representing seconds elapsed.
* **`hour`**
    * Hour that the trip started.
    * Integer in the range `[0, 23]`
* **`day_of_week`**
    * Integer in the range `[1, 7]`.
* **`is_big_tip`**
    * Whether the tip amount was greater than 20%.
    * Boolean `[True, False]`.

__Read, preprocess with Ray Data__

In [ ]:
ray.init(runtime_env={'pip':['xgboost-ray']})

In [ ]:
dataset = ray.data.read_parquet("s3://anonymous@anyscale-training-data/intro-to-ray-air/nyc_taxi_2021.parquet").repartition(16)

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

__Fit model with Ray Train__

In [ ]:
trainer = XGBoostTrainer(
    label_column="is_big_tip",
    scaling_config=ScalingConfig(num_workers=4, use_gpu=False),
    params={ "objective": "binary:logistic", },
    datasets={"train": train_dataset, "valid": valid_dataset},
    run_config=RunConfig(storage_path='/mnt/cluster_storage/')
)

result = trainer.fit()

__Optimize hyperparams with Ray Tune__

In [ ]:
tuner = Tuner(trainer, 
            param_space={'params' : {'max_depth': tune.randint(2, 12)}},
            tune_config=TuneConfig(num_samples=3, metric='train-logloss', mode='min'),
            run_config=RunConfig(storage_path='/mnt/cluster_storage/'))

checkpoint = tuner.fit().get_best_result().checkpoint

__Batch prediction__

In [ ]:
checkpoint.path

In [ ]:
! ls /mnt/cluster_storage/XGBoostTrainer_2023-08-29_14-57-29/XGBoostTrainer_0c8a1_00002_2_max_depth=10_2023-08-29_14-57-29/checkpoint_000000

In [ ]:
class OfflinePredictor:
    def __init__(self):
        import xgboost
        self._model = xgboost.Booster()
        self._model.load_model(checkpoint.path + '/model.json')
        #self._model = XGBoostPredictor.from_checkpoint(checkpoint)

    def __call__(self, batch):
        import xgboost
        import pandas as pd
        dmatrix = xgboost.DMatrix(pd.DataFrame(batch))    
        outputs = self._model.predict(dmatrix)
        return {"prediction": outputs}

In [ ]:
predicted_probabilities = valid_dataset.drop_columns(['is_big_tip']).map_batches(OfflinePredictor, compute=ray.data.ActorPoolStrategy(size=2))

In [ ]:
valid_dataset.drop_columns(['is_big_tip']).take_batch()

In [ ]:
predicted_probabilities.take_batch()

__Online prediction with Ray Serve__

In [ ]:
@serve.deployment
class OnlinePredictor:
    def __init__(self, checkpoint):
        import xgboost
        self._model = xgboost.Booster()
        self._model.load_model(checkpoint.path + '/model.json')        
        
    async def __call__(self, request: Request) -> dict:
        data = await request.json()
        data = json.loads(data)
        return {"prediction": self.get_response(data) }
    
    def get_response(self, data):
        import pandas as pd
        dmatrix = xgboost.DMatrix(pd.DataFrame(data, index=[0])) 
        return self._model.predict(dmatrix)

handle = serve.run(OnlinePredictor.bind(checkpoint=checkpoint))

In [ ]:
sample_input = valid_dataset.take(1)[0]
del(sample_input['is_big_tip'])
del(sample_input['__index_level_0__'])

requests.post("http://localhost:8000/", json=json.dumps(sample_input)).json()

In [ ]:
serve.shutdown()